In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
  
class TwitterClient(object):
    def __init__(self):
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
        consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
           
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets 
            for tweet in fetched_tweets:

                # empty dictionary for tweet data
                parsed_tweet = {}
  
                # tweet text
                parsed_tweet['text'] = tweet.text
                # tweet sentiment
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
      
                if tweet.retweet_count > 0:
                    # append retweet only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error if any exists
            print("Error : " + str(e))


  
def main():

    # TwitterClient Class object
    api = TwitterClient()

    query = input("Enter search keywaord: ")

    # get tweets
    tweets = api.get_tweets(query = query, count = 200)
  
    # picking positive tweets from tweets
    positive_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']

    # positive tweets %
    print("Positive tweets percentage: {r:2.2f} %".format(r=100*len(positive_tweets)/len(tweets)))

    # negative tweets 
    negative_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

    # negative tweets %
    print("Negative tweets percentage: {r:2.2f} %".format(r=100*len(negative_tweets)/len(tweets)))

    # neutral tweets %
    print("Neutral tweets percentage: {r:2.2f} %".format(r=100*(len(tweets) -(len( negative_tweets )+len( positive_tweets)))/len(tweets)))
  
    # top 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in positive_tweets[:10]:
        print(tweet['text'])
  
    # top 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in negative_tweets[:10]:
        print(tweet['text'])
  
if __name__ == "__main__":
    # main function
    main()